# CS4225/CS5425 Assignment 2

Name: Wu Tong

Matric Number: A0255954R

## 1. Data Preparation

### 1.1 Download Data (Citi Bike)

In [0]:
import urllib.request
import zipfile
import os

# Function download CitiBike data
def download_unzip(url, extract_to='/tmp/'):
    file_name = url.split('/')[-1]
    zip_path = os.path.join(extract_to, file_name)
    urllib.request.urlretrieve(url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)

In [0]:
# Call the function to download CitiBike data
# 2022 whole-year data
download_unzip('https://s3.amazonaws.com/tripdata/202201-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202202-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202203-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202204-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202205-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202206-citbike-tripdata.csv.zip')  # Typo!
download_unzip('https://s3.amazonaws.com/tripdata/202207-citbike-tripdata.csv.zip')  # Typo!
download_unzip('https://s3.amazonaws.com/tripdata/202208-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202209-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202210-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202211-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202212-citibike-tripdata.csv.zip')
# 2023 Jan-Jul data
download_unzip('https://s3.amazonaws.com/tripdata/202301-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202302-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202303-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202304-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202305-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202306-citibike-tripdata.csv.zip')
download_unzip('https://s3.amazonaws.com/tripdata/202307-citibike-tripdata.csv.zip')

In [0]:
%%sh
cd /tmp
mv 202206-citbike-tripdata.csv 202206-citibike-tripdata.csv
mv 202207-citbike-tripdata.csv 202207-citibike-tripdata.csv
ls

202201-citibike-tripdata.csv
202202-citibike-tripdata.csv
202203-citibike-tripdata.csv
202204-citibike-tripdata.csv
202205-citibike-tripdata.csv
202206-citibike-tripdata.csv
202207-citibike-tripdata.csv
202208-citibike-tripdata.csv
202209-citibike-tripdata.csv
202210-citibike-tripdata.csv
202211-citibike-tripdata.csv
202212-citibike-tripdata.csv
202301-citibike-tripdata.csv
202302-citibike-tripdata.csv
202303-citibike-tripdata.csv
202304-citibike-tripdata.csv
202305-citibike-tripdata.csv
202306-citibike-tripdata.csv
202307-citibike-tripdata.csv
Rserv
RtmpoqZO75
__MACOSX
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
load-from-jar5367716284729453345lib-diskcache.so
python_lsp_logs
systemd-private-d3be94e18cbd44fdb03a6d333e00f28e-systemd-logind.service-qxGSti
systemd-private-d3be94e18cbd44fdb03a6d333e00f28e-systemd-resolved.service-fRLkvs
tmp.oyJicEoAlm
tmpiqlg748x


#### 1.1.1 Read CitiBike data into Spark DF

In [0]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, min, to_date, datediff, to_timestamp, udf, count
from pyspark.sql.types import DoubleType, IntegerType, FloatType
import math

In [0]:
# Helper function
def haversine(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2):
        return None
    # Convert longtitude and latitude into radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371                        # Earth's average radius
    return c * r

spark.udf.register("haversine_udf", haversine, FloatType())


<function __main__.haversine(lon1, lat1, lon2, lat2)>

In [0]:
# Initialise a Spark session
spark = SparkSession.builder.appName("CitiBikeDataAggregation").getOrCreate()

# Add 2022 whole-year files and 2023 Jan-Jul files
months_2022 = [f"{month:02d}" for month in range(1, 13)]
months_2023 = [f"{month:02d}" for month in range(1, 8)]
file_list_2022 = [f'file:/tmp/2022{month}-citibike-tripdata.csv' for month in months_2022]
file_list_2023 = [f'file:/tmp/2023{month}-citibike-tripdata.csv' for month in months_2023]
file_list = file_list_2022 + file_list_2023
combined_df = None

for file_name in file_list:
    # Read data
    df = spark.read.csv(file_name, header=True, inferSchema=True)

    # Convert started_at and ended_at col into timestamp col
    df = df.withColumn('started_at', to_timestamp('started_at', 'yyyy/MM/dd HH:mm:ss'))
    df = df.withColumn('ended_at', to_timestamp('ended_at', 'yyyy/MM/dd HH:mm:ss'))

    # Calculate the trip distance and time
    df = df.withColumn('trip_distance', haversine_udf(
        col('start_lng'),
        col('start_lat'),
        col('end_lng'),
        col('end_lat')
    ))
    df = df.withColumn('trip_time', (col('ended_at').cast("long") - col('started_at').cast("long")) / 60) # In

    # Aggregate data for daily records
    daily_df = df.groupBy(to_date(col('started_at')).alias('datetime')).agg(
        count('*').alias('trip_count'),
        sum('trip_distance').alias('trip_distance'),
        sum('trip_time').alias('trip_time'),
    )
    combined_df = daily_df if combined_df is None else combined_df.union(daily_df)

# Aggregate again
final_df = combined_df.groupBy('datetime').agg(
    sum('trip_count').alias('trip_count'),
    sum('trip_distance').alias('trip_distance'),
    sum('trip_time').alias('trip_time'),
)

final_df = final_df.withColumnRenamed("_c0", "datetime") \
                   .withColumnRenamed("_c1", "trip_count") \
                   .withColumnRenamed("_c2", "trip_distance") \
                   .withColumnRenamed("_c3", "trip_time")

final_df.show()

+----------+----------+------------------+------------------+
|  datetime|trip_count|     trip_distance|         trip_time|
+----------+----------+------------------+------------------+
|2022-01-31|     22838| 36415.66340575871|381772.31666666653|
|2022-01-29|      2927| 3679.015376442131| 90255.73333333332|
|2022-01-15|     23717|32995.088187198315| 425115.9999999998|
|2022-01-09|     22088|34282.984465227644| 795394.7333333325|
|2022-01-18|     44045| 71586.66489926413|  632542.566666666|
|2022-01-10|     35717|56102.318114016285|1117036.0666666655|
|2022-01-08|     25129| 39177.86222436756|1151590.3500000008|
|2022-01-24|     42591| 68813.54992986325| 506331.0333333338|
|2022-01-23|     33817| 54528.93578195679| 512159.4666666667|
|2022-01-16|     21588| 31728.70841590948|390820.96666666644|
|2022-01-13|     51798| 89448.49425943883|  865903.033333333|
|2022-01-06|     46004|  75891.3533867981|1334009.9000000004|
|2022-01-03|     34126| 52264.02987379661| 1412850.700000001|
|2022-01

### 1.2 Read Data (Weather)

In [0]:
# Start a Spark session
spark = SparkSession.builder.appName("WeatherDataPreprocessing").getOrCreate()

# Read the weather dataset into a DataFrame
weather_df = spark.read.csv("/FileStore/weather_data.csv", header=True, inferSchema=True)

# Select the required attributes
selected_columns = [
    "datetime", "tempmax", "tempmin", "precip", "precipcover", 
    "snow", "snowdepth", "windspeed", "visibility", "uvindex"
]
weather_df = weather_df.select(selected_columns)

In [0]:
weather_df.show()

+----------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|  datetime|tempmax|tempmin|precip|precipcover|snow|snowdepth|windspeed|visibility|uvindex|
+----------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|2022-01-01|   13.5|   10.0|18.463|      54.17| 0.0|      0.0|     13.1|       8.7|      1|
|2022-01-02|   14.7|    3.6| 2.318|       12.5| 0.0|      0.0|     22.9|      11.8|      2|
|2022-01-03|    3.3|   -4.7|   0.0|        0.0| 0.0|      0.0|     26.1|      16.0|      1|
|2022-01-04|    1.3|   -6.9|   0.0|        0.0| 0.0|      0.0|     19.9|      16.0|      5|
|2022-01-05|    8.1|   -0.5| 5.745|      33.33| 0.0|      0.0|     20.5|      13.5|      1|
|2022-01-06|    5.2|    1.8|   0.0|        0.0| 7.4|      2.5|     21.3|      16.0|      4|
|2022-01-07|    1.4|   -3.0| 9.314|      33.33| 4.4|     10.4|     24.0|      10.1|      5|
|2022-01-08|   -1.2|   -6.2|   0.0|        0.0| 0.0|      8.2|     24.0|      16

In [0]:
# Check the number of missing column
from pyspark.sql.functions import col, count, when, isnull

missing_data_count = weather_df.select([count(when(isnull(c), c)).alias(c) for c in weather_df.columns])
missing_data_count.show()

+--------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|datetime|tempmax|tempmin|precip|precipcover|snow|snowdepth|windspeed|visibility|uvindex|
+--------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|       0|      0|      0|     0|          0|   0|        0|        0|         0|      0|
+--------+-------+-------+------+-----------+----+---------+---------+----------+-------+



In [0]:
# Handle the missing data
from pyspark.sql.functions import mean, col

# Contineous data
numerical_columns = ["tempmax", "tempmin", "precip", "precipcover", "snow", "snowdepth", "windspeed", "visibility", "uvindex"]

# Calculate mean to fill the missing value
for column in numerical_columns:
    mean_value = weather_df.select(mean(col(column))).collect()[0][0]
    weather_df = weather_df.na.fill({column: mean_value})

# Show the DataFrame to verify
weather_df.show()

+----------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|  datetime|tempmax|tempmin|precip|precipcover|snow|snowdepth|windspeed|visibility|uvindex|
+----------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|2022-01-01|   13.5|   10.0|18.463|      54.17| 0.0|      0.0|     13.1|       8.7|      1|
|2022-01-02|   14.7|    3.6| 2.318|       12.5| 0.0|      0.0|     22.9|      11.8|      2|
|2022-01-03|    3.3|   -4.7|   0.0|        0.0| 0.0|      0.0|     26.1|      16.0|      1|
|2022-01-04|    1.3|   -6.9|   0.0|        0.0| 0.0|      0.0|     19.9|      16.0|      5|
|2022-01-05|    8.1|   -0.5| 5.745|      33.33| 0.0|      0.0|     20.5|      13.5|      1|
|2022-01-06|    5.2|    1.8|   0.0|        0.0| 7.4|      2.5|     21.3|      16.0|      4|
|2022-01-07|    1.4|   -3.0| 9.314|      33.33| 4.4|     10.4|     24.0|      10.1|      5|
|2022-01-08|   -1.2|   -6.2|   0.0|        0.0| 0.0|      8.2|     24.0|      16

### 1.3 Extra Data

No extra data is being used.

### 1.4 Combine Data

In [0]:
from pyspark.sql.functions import year

# Join the DataFrames on the `datetime` column
combined_df = final_df.join(weather_df, "datetime")

# Save the combined DataFrame to DBFS
combined_df.write.format("parquet").save("/tmp/combined_data")

# Read the table from DBFS to ensure it's ready for modelling
combined_df = spark.read.format("parquet").load("/tmp/combined_data")

In [0]:
combined_df.show()

+----------+----------+------------------+------------------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|  datetime|trip_count|     trip_distance|         trip_time|tempmax|tempmin|precip|precipcover|snow|snowdepth|windspeed|visibility|uvindex|
+----------+----------+------------------+------------------+-------+-------+------+-----------+----+---------+---------+----------+-------+
|2022-01-31|     22838| 36415.66340575871|381772.31666666653|   -0.9|   -6.1|   0.0|        0.0| 0.0|      8.2|     12.1|      16.0|      6|
|2022-01-29|      2927| 3679.015376442131| 90255.73333333332|   -1.5|  -10.2|15.831|      66.67| 7.0|     12.9|     29.7|       6.0|      1|
|2022-01-15|     23717|32995.088187198315| 425115.9999999998|   -5.8|  -11.4|   0.0|        0.0| 0.0|      0.0|     22.4|      16.0|      5|
|2022-01-09|     22088|34282.984465227644| 795394.7333333325|    4.3|   -1.5| 2.765|       25.0| 0.0|      5.4|     22.3|      14.8|      1|
|2022-01-18| 

### 1.5 Filter the final table / datafram into training data and testing data

In [0]:
# Now, we filter the combined DataFrame into training and testing datasets
training_data = combined_df.filter(year("datetime") == 2022)
testing_data = combined_df.filter(year("datetime") == 2023)

# Assuming you want to save these to DBFS as well
training_data.write.format("parquet").save("/tmp/training_data")
testing_data.write.format("parquet").save("/tmp/testing_data")

# Verify the data count
print(f"Training Data Count: {training_data.count()}")
print(f"Testing Data Count: {testing_data.count()}")

Training Data Count: 365
Testing Data Count: 212


## 2. Build ML Pipeline using Training Data

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
spark = SparkSession.builder.appName("MLPipeline").getOrCreate()

### 2.1  Encode the categorical features

There is no features can be encoded

### 2.2 & 2.3 Select the revelant features & Scaling down the numerical features

The selection of the revelant features is partly done in data preprocessing stage, and more on this stage.

In [0]:
# Create a feature vector from the input features
feature_columns = [
    "tempmax", "tempmin", "precip", "precipcover", "snow",
    "snowdepth", "windspeed", "visibility", "uvindex", 
    "trip_distance", "trip_time"
]

In [0]:
# Assembler select the revelant features
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

### 2.4 Build ML model

In [0]:
# Use Random Forest to build the model
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="trip_count")
pipeline = Pipeline(stages=[assembler, scaler, rf])
model = pipeline.fit(training_data)

## 3. Evaluate ML Pipeline using Testing Data

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
predictions = model.transform(testing_data)
results_df = predictions.select(
    col("datetime"),
    col("prediction").alias("predicted_trip_count"),
    col("trip_count").alias("actual_trip_count")
)
results_df.show()

+----------+--------------------+-----------------+
|  datetime|predicted_trip_count|actual_trip_count|
+----------+--------------------+-----------------+
|2023-01-01|   60288.51366861246|            52561|
|2023-01-02|  63914.894886709735|            60369|
|2023-01-28|   69809.56798149788|            68844|
|2023-01-11|   59255.77927570346|            69327|
|2023-01-12|   52099.86510083667|            53216|
|2023-01-25|    39050.8324158758|            43222|
|2023-01-21|  58016.093561969115|            60243|
|2023-01-26|    60532.5170215664|            66678|
|2023-01-05|    71201.7191154671|            74016|
|2023-01-31|   53370.89711694243|            59258|
|2023-01-03|   52646.97099661912|            53591|
|2023-01-13|   63121.32245717246|            67539|
|2023-01-14|  46559.359639975235|            44968|
|2023-01-06|  62877.039757549566|            67023|
|2023-01-22|   41989.99571537206|            40698|
|2023-01-04|   75680.51359480638|            76968|
|2023-01-07|

In [0]:
evaluator = RegressionEvaluator(labelCol="trip_count", predictionCol="prediction")
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

MAE: 5421.270145175778
RMSE: 6534.847879410365
R-squared: 0.9536338661563291


In [0]:
model.save("/tmp/citibike_model")